# Xgboost Prediction

## Parameters Setting

In [79]:
# Data path
DATA_PATH = '/Users/vanessahuang/Desktop/BI_capstone/信義房屋/Xinyi_HousePrediction/prediction/0421_GroupTest/Data/0405(四版).csv'

# Total data used
FRACTION_SELECTED = 0.2

# K-means group number
MIN_CLUSTERS_KMEANS = 2
MAX_CLUSTERS_KMEANS = 20

# DBSCAN group range
EPS_RANGE_DBSCAN = (100, 100)
MIN_SAMPLES_RANGE_DBSCAN = (15, 15)

## K-means
會產出 `MIN_CLUSTERS_KMEANS` 到 `MAX_CLUSTERS_KMEANS` 共 (MAX_CLUSTERS_KMEANS - MIN_CLUSTERS_KMEANS)群的結果，並存在 **kmeans_result_dfs** 裏面。

In [16]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from Xinyi_Kmeans import kmeans_cluster_analysis

In [17]:
kmeans_result_dfs = kmeans_cluster_analysis(DATA_PATH, FRACTION_SELECTED, MIN_CLUSTERS_KMEANS, MAX_CLUSTERS_KMEANS)
for result_df in kmeans_result_dfs:
    print(result_df.head())

       土地移轉總面積平方公尺      移轉層次      總樓層數  建物移轉總面積平方公尺  建物現況格局-房  建物現況格局-廳  \
34457    -0.010714 -0.152531 -0.872621    -0.071598  0.412186  0.584395   
30727    -0.069220 -0.635776 -1.057031    -0.232169  0.412186 -0.600795   
36777     0.026336 -0.635776 -0.872621    -0.108507  0.412186  0.584395   
40045    -0.219061 -0.394154  0.602656    -0.136352 -0.203066 -0.600795   
12867     0.261768 -1.119022  0.602656     0.655776  1.027438  0.584395   

       建物現況格局-衛  建物現況格局-隔間  有無管理組織       總價元  ...  主要建材_磚造  主要建材_加強磚造  \
34457  0.442322   0.302105       0  20880000  ...        0          0   
30727 -0.445601   0.302105       0   8000000  ...        1          1   
36777 -0.445601   0.302105       0  16680000  ...        0          0   
40045 -0.445601   0.302105       1  15680000  ...        0          0   
12867  0.442322   0.302105       1  67300000  ...        0          0   

       主要建材_石造  主要建材_預力  主要建材_木造  主要建材_土造  主要建材_磚石造  主要建材_土磚石混合造  主要建材_壁式預鑄  \
34457        0        0        

In [178]:
kmeans_result_dfs[3]["分群結果_K-means_5"].value_counts()

分群結果_K-means_5
3    3438
2    2915
1    2847
0     436
4       2
Name: count, dtype: int64

## DBSCAN

會產出 `len(EPS_RANGE_DBSCAN)*len(MIN_SAMPLES_RANGE_DBSCAN)` 個資料，並存在 **DBSCAN_result_dfs** 裏面。

- **EPS**: 每個點「要搜尋的周圍範圍要多大」
- **min_samples**: 「有多少個點以上」才算密度夠高

In [70]:
from Xinyi_DBSCAN import dbscan_cluster_analysis

In [246]:
DBSCAN_result_dfs = dbscan_cluster_analysis(DATA_PATH, FRACTION_SELECTED, EPS_RANGE_DBSCAN, MIN_SAMPLES_RANGE_DBSCAN)
for result_df in DBSCAN_result_dfs:
    print(result_df.head())

       土地移轉總面積平方公尺      移轉層次      總樓層數  建物移轉總面積平方公尺  建物現況格局-房  建物現況格局-廳  \
36968    -0.188577  0.813959  0.233836    -0.082702  1.027438  0.584395   
2374      0.072649  0.089091 -0.503802     0.229376  1.027438  0.584395   
18662    -0.237352 -1.119022 -0.872621    -0.328642 -1.433569 -0.600795   
8166     -0.061599 -1.119022  0.418246     0.010866 -1.433569 -1.785984   
40298    -0.185646 -1.119022 -0.872621    -0.336910 -0.818318 -0.600795   

       建物現況格局-衛  建物現況格局-隔間  有無管理組織       總價元  ...  主要建材_磚造  主要建材_加強磚造  \
36968  0.442322   0.302105       1  26400000  ...        0          0   
2374   0.442322   0.302105       0  47000000  ...        0          0   
18662 -0.445601   0.302105       0  12750000  ...        0          0   
8166  -1.333524  -3.310108       1  13800000  ...        0          0   
40298 -0.445601   0.302105       0   6330000  ...        0          0   

       主要建材_石造  主要建材_預力  主要建材_木造  主要建材_土造  主要建材_磚石造  主要建材_土磚石混合造  主要建材_壁式預鑄  \
36968        0        0        

## Parameters determination – XGboost

In [20]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.model_selection import cross_val_score
from sklearn.utils import resample
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from xgboost import plot_importance
from matplotlib.ticker import FuncFormatter
import seaborn as sns
sns.set_style("whitegrid",{"font.sans-serif":['Microsoft JhengHei']})

### Functions: `split_data`, `hyperparameter_tuning`, `train_model`, `train_model_threshold`

In [21]:
# Function to split the data into training and testing sets: split_data
def split_data(clustered_name, targer_feature, test_size, random_state):
    X = clustered_name.drop([targer_feature], axis=1)
    y = clustered_name[targer_feature]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    return X_train, X_test, y_train, y_test

In [22]:
# Function for hyperparameter tuning: hyperparameter_tuning
def hyperparameter_tuning(cv, n_jobs, X_train, y_train):
    model_dbscan = xgb.XGBRegressor()
    param_grid = {
    'max_depth': [3, 4, 5],
    'learning_rate': [0.1, 0.01, 0.05],
    'n_estimators': [100, 200, 300]
    }
    grid_search = GridSearchCV(model_dbscan, param_grid, cv=cv, n_jobs=n_jobs)
    grid_search.fit(X_train, y_train)
    return grid_search.best_params_

In [34]:
# Function to train the model: train_model
def train_model(max_depth, learning_rate, n_estimators, X_train, y_train, X_test, y_test):
    model = xgb.XGBRegressor(max_depth=max_depth, learning_rate=learning_rate, n_estimators=n_estimators)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    mae = np.mean(np.abs(y_test - y_pred))
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    return mse, mae, rmse, r2

In [271]:
# Function to train the model: train_model_threshold
def train_model_threshold(max_depth, learning_rate, n_estimators, X_train, y_train, X_test, y_test, threshold):
    model = xgb.XGBRegressor(max_depth=max_depth, learning_rate=learning_rate, n_estimators=n_estimators)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    # Set if the difference between the predicted value and the actual value is less than 10%, it is correct
    y_pred = np.where(y_pred > (1 - threshold)* y_test, y_test, y_pred)
    y_pred = np.where(y_pred < (1 + threshold) * y_test, y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    mae = np.mean(np.abs(y_test - y_pred))
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    return mse, mae, rmse, r2

### K-means: 發現 `k=3` 的時候最好（r2 高, rmse 和 mse 小）

In [35]:
k_group = 2
for kmeans_data in kmeans_result_dfs:
    # Function to split the data into training and testing sets
    X_train_kmeans, X_test_kmeans, y_train_kmeans, y_test_kmeans = split_data(kmeans_data, '總價元', 0.2, 42)

    # Find the best hyperparameters for the model
    best_params_kmeans = hyperparameter_tuning(5, -1, X_train_kmeans, y_train_kmeans)

    # Train the model with the best hyperparameters
    mse_kmeans, mae_kmeans, rmse_kmeans, r2_kmeans = train_model(best_params_kmeans['max_depth'], best_params_kmeans['learning_rate'], best_params_kmeans['n_estimators'], X_train_kmeans, y_train_kmeans, X_test_kmeans, y_test_kmeans)
    print(f'Kmeans(k = {k_group}):')
    print('MSE:', mse_kmeans)
    print('MAE:', mae_kmeans)
    print('RMSE:', rmse_kmeans)
    print('R2:', r2_kmeans)
    print("="*30)

    k_group += 1

Kmeans(k = 2):
MSE: 117327894582292.05
MAE: 4448491.34906639
RMSE: 10831800.154281469
R2: 0.9045599344295832
Kmeans(k = 3):
MSE: 115159203814006.83
MAE: 4444723.969787344
RMSE: 10731225.643606924
R2: 0.9063240501998694
Kmeans(k = 4):
MSE: 120062364407652.61
MAE: 4486509.529029435
RMSE: 10957297.313099276
R2: 0.9023355871815407
Kmeans(k = 5):
MSE: 115470443592590.08
MAE: 4450328.704060028
RMSE: 10745717.453599367
R2: 0.9060708730250648
Kmeans(k = 6):
MSE: 197805881858736.4
MAE: 4831932.049176608
RMSE: 14064347.90023115
R2: 0.8390953285062935
Kmeans(k = 7):
MSE: 213620016918799.9
MAE: 4836182.446155342
RMSE: 14615745.513616467
R2: 0.826231362162695
Kmeans(k = 8):
MSE: 117764287029490.45
MAE: 4478115.290735521
RMSE: 10851925.498707151
R2: 0.9042049521474641
Kmeans(k = 9):
MSE: 115252660425628.66
MAE: 4457707.02414257
RMSE: 10735579.184451515
R2: 0.9062480281662948
Kmeans(k = 10):
MSE: 125732539938766.2
MAE: 4471957.816029402
RMSE: 11213052.213325603
R2: 0.8977231978907255
Kmeans(k = 11):


### DBSCAN: 手動測試發現 `EPS=3`, `MIN_SAMPLE=30` 的時候效果很好

> ##### 手動測試

In [248]:
for dbscan_data in DBSCAN_result_dfs:
    # Function to split the data into training and testing sets
    X_train_dbscan, X_test_dbscan, y_train_dbscan, y_test_dbscan = split_data(dbscan_data, '總價元', 0.2, 42)

    # Find the best hyperparameters for the model
    best_params_dbscan = hyperparameter_tuning(5, -1, X_train_dbscan, y_train_dbscan)

    # Train the model with the best hyperparameters
    mse_dbscan, mae_dbscan, rmse_dbscan, r2_dbscan = train_model(best_params_dbscan['max_depth'], best_params_dbscan['learning_rate'], best_params_dbscan['n_estimators'], X_train_dbscan, y_train_dbscan, X_test_dbscan, y_test_dbscan)

    # Print the results with its corresponding eps and min_samples
    print(f'DBSCAN(eps = {EPS_RANGE_DBSCAN[0]}, min_samples = {MIN_SAMPLES_RANGE_DBSCAN[0]}):')
    print('MSE:', mse_dbscan)
    print('MAE:', mae_dbscan)
    print('RMSE:', rmse_dbscan)
    print('R2:', r2_dbscan)
    print("="*30)

DBSCAN(eps = 3, min_samples = 30):
MSE: 114490144855324.14
MAE: 4475886.221715595
RMSE: 10700006.76893824
R2: 0.9071832358275697


> ##### 有很多不同的資料及回圈測試（太多了 跑不動）

In [63]:
eps_values = list(range(EPS_RANGE_DBSCAN[0], EPS_RANGE_DBSCAN[1] + 1))
min_samples_values = list(range(MIN_SAMPLES_RANGE_DBSCAN[0], MIN_SAMPLES_RANGE_DBSCAN[1] + 1))

performance_results = []

from itertools import product
parameter_combinations = list(product(eps_values, min_samples_values))

for index, dbscan_data in enumerate(DBSCAN_result_dfs):
    eps, min_samples = parameter_combinations[index]

    # Function to split the data into training and testing sets
    X_train_dbscan, X_test_dbscan, y_train_dbscan, y_test_dbscan = split_data(dbscan_data, '總價元', 0.2, 42)

    # Find the best hyperparameters for the model
    best_params_dbscan = hyperparameter_tuning(5, -1, X_train_dbscan, y_train_dbscan)
    
    # Train the model with the best hyperparameters
    mse_dbscan, mae_dbscan, rmse_dbscan, r2_dbscan = train_model(best_params_dbscan['max_depth'], best_params_dbscan['learning_rate'], best_params_dbscan['n_estimators'], X_train_dbscan, y_train_dbscan, X_test_dbscan, y_test_dbscan)
    
    # Store the performance metrics along with the corresponding DBSCAN parameters
    performance_results.append((eps, min_samples, mse_dbscan, mae_dbscan, rmse_dbscan, r2_dbscan))
    
    # Print the results with its corresponding eps and min_samples
    print(f'DBSCAN(eps = {eps}, min_samples = {min_samples}):')
    print('MSE:', mse_dbscan)
    print('MAE:', mae_dbscan)
    print('RMSE:', rmse_dbscan)
    print('R2:', r2_dbscan)
    print("="*30)

DBSCAN(eps = 2, min_samples = 2):
MSE: 124193120284882.75
MAE: 4186914.395201938
RMSE: 11144196.70882037
R2: 0.8796438893513137
DBSCAN(eps = 2, min_samples = 3):
MSE: 131964845171293.05
MAE: 4371404.984773348
RMSE: 11487595.273654668
R2: 0.8721122758592432
DBSCAN(eps = 2, min_samples = 4):
MSE: 123428648320378.7
MAE: 4186542.344075182
RMSE: 11109844.657796917
R2: 0.8803847425655384
DBSCAN(eps = 2, min_samples = 5):
MSE: 122336230788513.55
MAE: 4165225.9003072972
RMSE: 11060570.997399436
R2: 0.8814434093019738
DBSCAN(eps = 2, min_samples = 6):
MSE: 122895772900187.88
MAE: 4171603.681719888
RMSE: 11085836.590000227
R2: 0.8809011545285147
DBSCAN(eps = 2, min_samples = 7):
MSE: 122166195119862.97
MAE: 4136880.8662404073
RMSE: 11052881.756350376
R2: 0.8816081916321334
DBSCAN(eps = 2, min_samples = 8):
MSE: 123616039407135.47
MAE: 4163207.456197971
RMSE: 11118275.01940546
R2: 0.8802031410217449
DBSCAN(eps = 2, min_samples = 9):
MSE: 122871102574089.52
MAE: 4163343.2206116677
RMSE: 11084723.8

KeyboardInterrupt: 

## Predicting Model: whole dataset

### K-means

In [279]:
# 取得 k = 3 的分群結果
kmeans_result_dfs = kmeans_cluster_analysis(DATA_PATH, 1, 3, 3)

# Function to split the data into training and testing sets
X_train_kmeans, X_test_kmeans, y_train_kmeans, y_test_kmeans = split_data(kmeans_result_dfs[0], '總價元', 0.2, 42)

# Find the best hyperparameters for the model
best_params_kmeans = hyperparameter_tuning(5, -1, X_train_kmeans, y_train_kmeans)

In [280]:
# Train the model with function: train_model
mse_kmeans, mae_kmeans, rmse_kmeans, r2_kmeans = train_model(best_params_kmeans['max_depth'], best_params_kmeans['learning_rate'], best_params_kmeans['n_estimators'], X_train_kmeans, y_train_kmeans, X_test_kmeans, y_test_kmeans)
print(f'Kmeans with train_model:')
print('RMSE:', f"{rmse_kmeans:,.2f}", "元")
print('R2:', f"{r2_kmeans:.2f}")

Kmeans with train_model:
RMSE: 33,463,412.22 元
R2: 0.81


### DBSCAN

In [266]:
# 取得 eps = 3, min_samples = 30 的分群結果
dbscan_data = dbscan_cluster_analysis(DATA_PATH, 1, (3,3), (30,30))

# Function to split the data into training and testing sets
X_train_dbscan, X_test_dbscan, y_train_dbscan, y_test_dbscan = split_data(dbscan_data[0], '總價元', 0.2, 42)

# Find the best hyperparameters for the model
best_params_dbscan = hyperparameter_tuning(5, -1, X_train_dbscan, y_train_dbscan)

In [268]:
# Train the model with function: train_model
mse_dbscan, mae_dbscan, rmse_dbscan, r2_dbscan = train_model(best_params_dbscan['max_depth'], best_params_dbscan['learning_rate'], best_params_dbscan['n_estimators'], X_train_dbscan, y_train_dbscan, X_test_dbscan, y_test_dbscan)

print("DBSCAN with train_model")
print('RMSE:', f"{rmse_dbscan:,.2f}", "元")
print('R2:', f"{r2_dbscan:.2f}")

DBSCAN with train_model
RMSE: 16,030,844.96 元
R2: 0.92
